**IMPORT LIBRARIES**

In [5]:
from datetime import datetime, timedelta

import plotly.graph_objs as go
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import satSimulationMagField as sim

import parallel_test4 as simulation
import plotMagneticField as hplot

**Run Satellite Orbit Propagator**

In [6]:
# TLE of the satellite
line1 = "1 00000U 20000A   25015.00000000  .00000000  00000-0  00000-0 0  9991"
line2 = "2 00000  97.3400 313.1310 0000000  0.0000  0.0000 15.09556778000000"

# Simulation parameters
start_date = datetime(2025, 1, 15)
end_date = start_date + timedelta(days=100)
time_step = timedelta(hours=0.08)

# Run the simulation
sim.initialize_satellite(line1, line2)
df = sim.simulate_satellite(start_date, end_date, time_step)

# Save results to CSV
df.to_csv("satellite_propagation_results.csv", index=False)
print("Results saved to 'satellite_propagation_results.csv'")

Results saved to 'satellite_propagation_results.csv'


In [7]:
df = pd.read_csv('satellite_propagation_results.csv')
values = sim.calculate_max_min_values(df, 'ECI')

Max Value Bx ECI (nT): 45839.86895741893 nT, Min Value Bx ECI (nT): -45342.45177870458 nT
Max Value By ECI (nT): 46208.256178266194 nT, Min Value By ECI (nT): -34046.21391202095 nT
Max Value Bz ECI (nT): 44894.94890550539 nT, Min Value Bz ECI (nT): -45448.63680014032 nT


In [8]:
# Initialize coil parameters
X_coil = simulation.CoilParameters(1, 0.5, 36)
Y_coil = simulation.CoilParameters(1, 0.5, 36)
Z_coil = simulation.CoilParameters(1, 0.5, 36)
#X_coil = test_3.CoilParameters(0.98083217, 0.95992063, 0.00128399)

# Current coil simulation
I  = np.array([1,2,3])
Ax = np.eye(3)
Ay = np.array([[0, -1,  0], [1,  0,  0], [0,  0,  1]])
Az = np.array([[0,  0, -1], [0,  1,  0], [1,  0,  0]])

# X coil simulation
grid_length_size = 0.1
num_seg = 100

In [10]:
# Definir los argumentos adicionales
Ax = np.eye(3)  # Identity matrix
target_bx = 45839.86895741893  # Desired target for Bx
target_by = 45839.86895741893  # Desired target for By
target_bz = 45839.86895741893  # Desired target for Bz

# Bounds for the variables
my_bounds = [(0.4, 1), (0.4, 1)]  # Bounds for length and distance

# Tolerancia de 3 cm
tolerance_cm = 0.03  # 3 cm en unidades de metros

# Restricción dinámica: length no puede estar en un rango prohibido
def constraint_length_not_in_range(variables, forbidden_value, tolerance=0.1):
    """
    Restricción que asegura que `length` no esté dentro de [forbidden_value - tolerance, forbidden_value + tolerance].
    """
    length = variables[0]  # Extraer length de las variables
    return min(length - (forbidden_value + tolerance), (forbidden_value - tolerance) - length)

# First optimization (X)
constraints_x = []  # No forbidden value initially

initial = [1,0.4]

length_x, distance_x = simulation.optimization_fun(Ax, initial, target_bx, my_bounds, constraints_x)

# Update forbidden value for Y optimization
forbidden_value = length_x
constraints_y = [
    {
        "type": "ineq",
        "fun": constraint_length_not_in_range,
        "args": (forbidden_value, tolerance_cm)  # Rango alrededor de forbidden_value
    }
]

initial = [length_x, distance_x]
# Second optimization (Y)
length_y, distance_y = simulation.optimization_fun(Ax, initial, target_by, my_bounds, constraints_y)

# Update forbidden values for Z optimization
forbidden_value1 = length_x
forbidden_value2 = length_y
constraints_z = [
    {
        "type": "ineq",
        "fun": constraint_length_not_in_range,
        "args": (forbidden_value1, tolerance_cm)  # Rango alrededor de forbidden_value1
    },
    {
        "type": "ineq",
        "fun": constraint_length_not_in_range,
        "args": (forbidden_value2, tolerance_cm)  # Rango alrededor de forbidden_value2
    }
]
initial = [length_y, distance_y]
# Third optimization (Z)
length_z, distance_z = simulation.optimization_fun(Ax, initial, target_bz, my_bounds, constraints_z)

TypeError: optimization_fun() takes 4 positional arguments but 5 were given

In [ ]:
##Generar espiras
spire1_x_s, spire2_x_s = simulation.square_spires(Ax, X_coil.h, X_coil.a, num_seg)
spire1_y_s, spire2_y_s = simulation.square_spires(Ay, Y_coil.h, Y_coil.a, num_seg)
spire1_z_s, spire2_z_s = simulation.square_spires(Az, Z_coil.h, Z_coil.a, num_seg)

f0 = None
f0 = hplot.plot_spires(f0, spire1_x_s, spire2_x_s, color='black')
f0 = hplot.plot_spires(f0, spire1_y_s, spire2_y_s, color='red')
f0 = hplot.plot_spires(f0, spire1_z_s, spire2_z_s, color='blue')
# Display the final figure
f0.update_layout(
        title="Gráficos de Superficie 3D - Todos los Planos",
)
f0.show()

**Orbit propagator plots**

In [ ]:
hplot.plot_orbit(df, 'ECI')

In [ ]:
hplot.plot_magField_time(df, 'NED')